In [70]:
from typing import TypedDict
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import AnyMessage, HumanMessage,ToolMessage
# 1. 定义自定义 State
class CustomState(AgentState):
    user_name: str

model = ChatAnthropic(
    model="mimo-v2-flash",
    anthropic_api_key="sk-c01l9de7awq589w1b0oxtwo7mzizvi487jvtaio5lbg8sabb",
    base_url="https://api.xiaomimimo.com/anthropic"
)
# 2. 定义更新 State 的工具
@tool
def update_user_name(new_name: str, runtime: ToolRuntime) -> Command:
    """更新用户的名字。"""
    # 返回 Command(update=...) 来修改 State
    tool_msg = ToolMessage(
        content=f"Successfully recorded {len(new_name)} evidence(s): {', '.join(new_name[:2])}{'...' if len(new_name) > 2 else ''}",
        tool_call_id=runtime.tool_call_id
    )
    return Command(update={"user_name": new_name,"messages":[tool_msg]})

# 3. 创建 Agent 时传入 state_schema
agent = create_agent(
    model=model,
    tools=[update_user_name],
    state_schema=CustomState
)

# 4. 调用
result = agent.invoke({
    "messages": [{"role": "user", "content": "把我的名字改为张三"}],
    "user_name": "匿名用户"
})
print(result.get("user_name"))

张三


In [2]:
result.get("user_name")

'张三'

In [67]:
"""分析并升级用户的代码示例"""

import operator
from typing import Annotated
from langchain.tools import tool
from langchain.agents.middleware import AgentState, AgentMiddleware
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.types import Command
from deepagents import create_deep_agent, SubAgent


# ========================================
# ❌ 原代码的问题分析
# ========================================

"""
问题1: context_schema 参数不起作用
  - create_deep_agent 的 context_schema 参数不会正确添加自定义字段到 state
  - 解决: 必须使用 middleware.state_schema

问题2: Subagent 没有包含自定义 state middleware
  - Subagent 需要同样的 middleware 才能返回自定义字段
  - 解决: 在 SubAgent 的 middleware 中添加

问题3: save_evidences 的参数设计问题
  - 参数是 evidences: list[str]，意味着调用者需要传入整个列表
  - 这样每次都是替换而不是追加
  - 解决: 改为单个 evidence: str，然后在内部追加

问题4: Tool 返回的是完整列表而非增量
  - 如果多个 tool 并行调用，会有冲突
  - 解决: 使用 Annotated[list[str], operator.add] 让 LangGraph 自动合并
"""


# ========================================
# ✅ 正确的升级方案
# ========================================

# 步骤1: 定义自定义 State（使用 Annotated 处理多值更新）
class Context(AgentState):
    evidences: Annotated[list[str], operator.add]


# 步骤2: 创建 Middleware 来添加 state_schema
class ContextMiddleware(AgentMiddleware):
    """扩展 State 的 Middleware"""
    state_schema = Context


# 步骤3: 修改 Tool - 追加单个证据而非整个列表
@tool
def save_evidence(evidence: str, runtime: ToolRuntime) -> Command:
    """
    保存单条证据。

    Args:
        evidence: 单条证据内容
    """
    current = runtime.state.get("evidences", [])
    print(f"[save_evidence] 当前证据数: {len(current)} -> 新增: {evidence}")

    tool_msg = ToolMessage(
        content=f"已保存证据: {evidence}",
        tool_call_id=runtime.tool_call_id
    )

    # 返回新列表（Annotated 会让 operator.add 自动合并多个更新）
    return Command(update={
        "evidences": [evidence],  # 只返回新增的证据，LangGraph 会自动合并
        "messages": [tool_msg]
    })


# 步骤4: 创建 Subagent（添加 ContextMiddleware）
subagent = SubAgent(
    name="medical_specialist",
    description="负责具体文献检索与证据保存的助理。",
    system_prompt="""你是一个医学专家助理。
当用户提供证据时，必须使用 save_evidence 工具逐条保存每条证据。
重要：每条证据都要单独调用一次 save_evidence！""",
    tools=[save_evidence],
    model=model,
    middleware=[ContextMiddleware()],  # ← 关键！Subagent 也需要这个 middleware
)


# 步骤5: 创建 Main Agent（添加 ContextMiddleware，移除 context_schema）
main_agent = create_deep_agent(
    name="main_agent",
    system_prompt="你是一个主智能体，负责协调整个诊断流程。",
    subagents=[subagent],
    tools=[save_evidence],  # Main agent 也可以直接调用这个工具
    middleware=[ContextMiddleware()],  # ← 使用 middleware 而非 context_schema
    model=model
)


# ========================================
# 📊 使用示例
# ========================================

# 运行 agent
result = main_agent.invoke({
    "messages": [HumanMessage(content="患者有以下症状：头痛、发烧、咳嗽")],
    "evidences": [],  # 初始化为空列表
})

# 查看结果
print(f"\n收集到的证据数量: {len(result['evidences'])}")
for i, evidence in enumerate(result['evidences'], 1):
    print(f"  {i}. {evidence}")


# ========================================
# 🔄 如果想保留原工具签名（批量保存）
# ========================================

# 如果确实需要批量保存，可以这样修改：
@tool
def save_evidences_batch(evidences: list[str], runtime: ToolRuntime) -> Command:
    """
    批量保存多条证据。

    Args:
        evidences: 要保存的证据列表
    """
    print(f"[save_evidences_batch] 批量保存 {len(evidences)} 条证据")
    print(1111111111111111)
    tool_msg = ToolMessage(
        content=f"已保存 {len(evidences)} 条证据",
        tool_call_id=runtime.tool_call_id
    )

    print("evidences", evidences)
    # 返回整个列表（Annotated 的 operator.add 会处理合并）
    return Command(update={
        "evidences": evidences,  # 返回完整列表
        "messages": [tool_msg]
    })


# ========================================
# ⚠️ 重要注意事项
# ========================================

"""
1. Middleware 必须同时添加到 Main Agent 和 Subagent
   - 否则 Subagent 返回的 evidences 会被丢弃

2. Annotated[list[str], operator.add] 的作用
   - 允许多个 tool 并行调用时自动合并列表
   - 等价于: evidences = evidences1 + evidences2 + ...

3. 返回 [evidence] 还是 evidences
   - 返回 [evidence]（单元素列表）: operator.add 会自动合并
   - 返回 evidences（完整列表）: 适合批量操作
   - 两者都可以，关键是使用 Annotated

4. 不要在 tool 中原地修改列表
   - ❌ current.append(evidence)  # 原地修改
   - ✅ return {"evidences": [evidence]}  # 返回新值
"""


DEBUG: Loading 6 middlewares for agent: main_agent
[save_evidence] 当前证据数: 0 -> 新增: 症状分析：头痛、发烧、咳嗽 - 这是一组常见的呼吸道感染相关症状组合
[save_evidence] 当前证据数: 1 -> 新增: 流感诊断要点：突发高热、剧烈头痛、全身肌肉酸痛、干咳，白细胞计数正常或偏低，淋巴细胞比例升高
[save_evidence] 当前证据数: 2 -> 新增: COVID-19典型症状：发热、干咳、乏力，部分患者出现嗅觉味觉减退或丧失，重症可出现呼吸困难、低氧血症
[save_evidence] 当前证据数: 3 -> 新增: 诊断流程建议：1.详细病史询问+体格检查；2.血常规+CRP；3.胸部影像学；4.病原学检测（流感/新冠等）；5.必要时痰培养、血气分析

收集到的证据数量: 4
  1. 症状分析：头痛、发烧、咳嗽 - 这是一组常见的呼吸道感染相关症状组合
  2. 流感诊断要点：突发高热、剧烈头痛、全身肌肉酸痛、干咳，白细胞计数正常或偏低，淋巴细胞比例升高
  3. COVID-19典型症状：发热、干咳、乏力，部分患者出现嗅觉味觉减退或丧失，重症可出现呼吸困难、低氧血症
  4. 诊断流程建议：1.详细病史询问+体格检查；2.血常规+CRP；3.胸部影像学；4.病原学检测（流感/新冠等）；5.必要时痰培养、血气分析


'\n1. Middleware 必须同时添加到 Main Agent 和 Subagent\n   - 否则 Subagent 返回的 evidences 会被丢弃\n\n2. Annotated[list[str], operator.add] 的作用\n   - 允许多个 tool 并行调用时自动合并列表\n   - 等价于: evidences = evidences1 + evidences2 + ...\n\n3. 返回 [evidence] 还是 evidences\n   - 返回 [evidence]（单元素列表）: operator.add 会自动合并\n   - 返回 evidences（完整列表）: 适合批量操作\n   - 两者都可以，关键是使用 Annotated\n\n4. 不要在 tool 中原地修改列表\n   - ❌ current.append(evidence)  # 原地修改\n   - ✅ return {"evidences": [evidence]}  # 返回新值\n'

In [64]:
result = main_agent.invoke({"messages":[HumanMessage(content="让子智能体测试下是否可以使用save_evidences工具")],"evidences":[]})

[save_evidence] 当前证据数: 0 -> 新增: 这是一条测试证据，用于验证工具是否正常工作


In [69]:
result

{'messages': [HumanMessage(content='患者有以下症状：头痛、发烧、咳嗽', additional_kwargs={}, response_metadata={}, id='eef889f9-0097-436d-a4a6-cac1f20a60c3'),
  AIMessage(content=[{'text': '我来帮您分析这些症状。基于患者出现的头痛、发烧和咳嗽症状，我需要进行医学分析。', 'type': 'text'}, {'id': 'call_920837655d714b55900366da', 'input': {'description': '分析头痛、发烧、咳嗽症状的可能病因和诊断建议', 'subagent_type': 'medical_specialist'}, 'name': 'task', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': '527bbb2cdbbb47c399816f3a45561d50', 'model': 'mimo-v2-flash', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': None, 'cache_creation_input_tokens': None, 'cache_read_input_tokens': None, 'input_tokens': 5225, 'output_tokens': 70, 'server_tool_use': None, 'service_tier': None}, 'model_name': 'mimo-v2-flash', 'model_provider': 'anthropic'}, name='main_agent', id='lc_run--019b6dd3-7e0f-7f00-9e6d-9b2926f46379-0', tool_calls=[{'name': 'task', 'args': {'description': '分析头痛、发烧、咳嗽症状的可能病因和诊断建议', 'subagent_type': 'medical_spec

In [68]:
evidence

'诊断流程建议：1.详细病史询问+体格检查；2.血常规+CRP；3.胸部影像学；4.病原学检测（流感/新冠等）；5.必要时痰培养、血气分析'

In [74]:
import time
from typing import Annotated
import operator
from langchain.agents.middleware import AgentState, AgentMiddleware
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.types import Command

# 1. 定义 State
class Context(AgentState):
    evidences: Annotated[list[str], operator.add]

# 2. 定义 Middleware（带调试输出）
class DebugContextMiddleware(AgentMiddleware):
    state_schema = Context

    def before_agent(self, state, runtime):
        print("\n" + "="*60)
        print("🚀 [before_agent] Agent 开始执行")
        print(f"   当前 evidences: {state.get('evidences', [])}")
        return None

    def before_model(self, state, runtime):
        print("🤖 [before_model] 准备调用 LLM")
        return None

    def wrap_model_call(self, request, handler):
        print("⚡ [wrap_model_call] 开始")
        start = time.time()

        # 调用实际模型
        response = handler(request)

        elapsed = time.time() - start
        print(f"⚡ [wrap_model_call] 完成 (耗时: {elapsed:.2f}s)")
        return response

    def after_model(self, state, runtime):
        last_msg = state["messages"][-1]
        if hasattr(last_msg, 'tool_calls'):
            print(f"🛠️  [after_model] 模型调用了 {len(last_msg.tool_calls)} 个工具")
        return None

    def after_agent(self, state, runtime):
        print(f"✅ [after_agent] Agent 完成")
        print(f"   最终 evidences: {state.get('evidences', [])}")
        print("="*60 + "\n")
        return None

# 3. 创建并运行 agent
agent = create_deep_agent(
    middleware=[DebugContextMiddleware()],
    subagents=[subagent],  # subagent 也需要这个 middleware！
    model=model,
)

# 运行时会看到完整的执行流程
result = agent.invoke({
    "messages": [HumanMessage("保存证据：头痛")],
    "evidences": [],
})

DEBUG: Loading 6 middlewares for agent: None

🚀 [before_agent] Agent 开始执行
   当前 evidences: []
🤖 [before_model] 准备调用 LLM
⚡ [wrap_model_call] 开始
⚡ [wrap_model_call] 完成 (耗时: 5.75s)
🛠️  [after_model] 模型调用了 1 个工具
🤖 [before_model] 准备调用 LLM
⚡ [wrap_model_call] 开始
⚡ [wrap_model_call] 完成 (耗时: 1.36s)
🛠️  [after_model] 模型调用了 1 个工具
🤖 [before_model] 准备调用 LLM
⚡ [wrap_model_call] 开始
⚡ [wrap_model_call] 完成 (耗时: 0.69s)
🛠️  [after_model] 模型调用了 1 个工具
🤖 [before_model] 准备调用 LLM
⚡ [wrap_model_call] 开始
⚡ [wrap_model_call] 完成 (耗时: 10.22s)
🛠️  [after_model] 模型调用了 1 个工具
🤖 [before_model] 准备调用 LLM
⚡ [wrap_model_call] 开始
⚡ [wrap_model_call] 完成 (耗时: 12.48s)
🛠️  [after_model] 模型调用了 0 个工具
✅ [after_agent] Agent 完成
   最终 evidences: []

